This Pyton script reads a sql query from Snowflake, creates a pandas dataframe and then writes the dataframe to the hometeam-clinical-data bucket on AWS S3. Ed user needs to pastse custom sql query and define the S3 path.

In [ ]:
import snowflake.connector as sf
import time
import os
import boto3
import pandas as pd
import csv


In [ ]:
# log
start_time = time.time()
log = []


In [ ]:
# Snowflake credentials stored in environment variables

username = os.getenv("Snowflake_User")
password = os.getenv("Snowflake_password")
account = os.getenv("Snowflake_account")

# Define warehouse, if neccessary
warehouse = "DEVELOPER_BASIC"

# Define Database, if not defined in SQL request
# database = 'VESTA_STAGING'

# Create connection object for Snowflake connection
conn = sf.connect(
    user=username, password=password, account=account, warehouse=warehouse
)

# Execution function
def execute_query(connection, query):
    cursor = connection.cursor()
    cursor.execute(query)
    cursor.close


try:
    # If defining a database, uncomment code set and add database in connection parameter
    # sql = 'use {}'.format(database)
    # execute_query(conn,sql)1011

    # Define warehouse to use in Snowflake
    sql = "use warehouse {}".format(warehouse)
    execute_query(conn, sql)

    print("Successful Connection")

    # Query to Snowflake
    sql = """
            SELECT *
            FROM
            VESTA_DEVELOPMENT.ANALYST_SANDBOX.VIP_SCORING
            WHERE CLNT = 'UHC_NE'
            AND DATA_DATE_START = '202209' limit 10 """
            
    cursor = conn.cursor()
    cursor.execute(sql)

    # Dataframe creation
    df = pd.DataFrame.from_records(
        iter(cursor), columns=[x[0] for x in cursor.description]
    )

    print("Successful DataFrame Created")

    cursor.close

except Exception as e:
    print(e)

finally:
    conn.close

df

In [ ]:
#Create Connection Object for Connecting to AWS

try:
    s3 = boto3.resource(
        service_name='s3',
        region_name='us-east-1',
        aws_access_key_id=os.getenv('aws_access_key_id'),
        aws_secret_access_key=os.getenv('aws_secret_access_key'))
    print('AWS connection object created at ' + time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(time.time())))
    log.append(
        'AWS connection object created at ' + time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(time.time())))
except Exception as e:
    print('There was an error with creating the AWS connection object - ' + str(e))
    log.append('There was an error with creating the AWS connection object - ' + str(e))


In [ ]:
# write df as csv to AWS bucket

df.to_csv('s3://hometeam-clinical-data/test.csv', index=False)